In [ ]:
%pip install geopandas matplotlib

In [ ]:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/RoadWorksAnalyzer/data/

/content/drive/.shortcut-targets-by-id/1llXkQFhfIsRiSyYeqMtBZUykMH-b08ey/RoadWorksAnalyzer/data


In [ ]:
import re

import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import LineString


import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Moscow-data-2024-12-05.csv', sep=';')
df.head()

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,Descr,Geometry_type,Geometry_point,Geometry_line,geoData,geodata_center,Unnamed: 17
0,Идентификатор,Местоположение,global_id,Административный округ,Район,Тип события,Наименование события,Инициатор,Дата и время начала перекрытия,Дата и время окончания перекрытия,Количество перекрытых полос,Дополнительная информация,Тип геометрии,Координаты для точечных объектов,Координаты для линейных объектов,geoData,geodata_center,NaN
1,20759,"Фрунзенская наб., д.8",2639121078,Центральный административный округ,район Хамовники,Ремонтные работы,Работы ДЖКХ,NaN,19.02.2024,20.05.2025,1,NaN,line,NaN,"37.595787444884735,55.73287012908498;37.596229...","{coordinates=[[[37.595787445, 55.732870129], [...",NaN,NaN
2,20890,ул. Большая Академическая (ул. Валаамская - ул...,2639494326,Северный административный округ,Тимирязевский район,Ремонтные работы,Работы коммерческих организаций,NaN,15.01.2024,08.12.2024,1,NaN,line,NaN,"37.56010003387928,55.84447126979461;37.5600074...","{coordinates=[[[37.560100034, 55.84447127], [3...",NaN,NaN
3,20659,ул. Новослободская (в сторону области) (ул. Па...,2639494348,Центральный административный округ,Тверской район,Ремонтные работы,Работы коммерческих организаций,NaN,29.05.2024,19.11.2024,1,NaN,line,NaN,"37.597433083289545,55.7834360473714;37.5972211...","{coordinates=[[[37.597433083, 55.783436047], [...",NaN,NaN
4,20301,"ул. Маршала Тухачевского, д.15 - д.14, к.1",2639494385,Северо-Западный административный округ,район Хорошёво-Мнёвники,Ремонтные работы,Работы ДЖКХ,NaN,07.02.2024,30.06.2024,1,NaN,line,NaN,"37.48624339699745,55.78197922246133;37.4867811...","{coordinates=[[[37.486243397, 55.781979222], [...",NaN,NaN


##**Первоначальная очистка данных**

Удаление признака c невалидным именем

In [ ]:
df = df.drop('Unnamed: 17', axis=1)

Проверка признаков на Nan

In [ ]:
df.isna().sum()

,0
Ext_id,0
Address,0
global_id,0
AdmArea,0
District,0
Event_type,0
Name,0
Requester,433
TS_from,0
TS_to,0


Признаки Descr, Geometry_point, geodata_center имеют только Nan значения. Удалим их

In [ ]:
df = df.drop(['Descr', 'Geometry_point', 'geodata_center'], axis=1)
df.head()

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,Geometry_type,Geometry_line,geoData
0,Идентификатор,Местоположение,global_id,Административный округ,Район,Тип события,Наименование события,Инициатор,Дата и время начала перекрытия,Дата и время окончания перекрытия,Количество перекрытых полос,Тип геометрии,Координаты для линейных объектов,geoData
1,20759,"Фрунзенская наб., д.8",2639121078,Центральный административный округ,район Хамовники,Ремонтные работы,Работы ДЖКХ,NaN,19.02.2024,20.05.2025,1,line,"37.595787444884735,55.73287012908498;37.596229...","{coordinates=[[[37.595787445, 55.732870129], [..."
2,20890,ул. Большая Академическая (ул. Валаамская - ул...,2639494326,Северный административный округ,Тимирязевский район,Ремонтные работы,Работы коммерческих организаций,NaN,15.01.2024,08.12.2024,1,line,"37.56010003387928,55.84447126979461;37.5600074...","{coordinates=[[[37.560100034, 55.84447127], [3..."
3,20659,ул. Новослободская (в сторону области) (ул. Па...,2639494348,Центральный административный округ,Тверской район,Ремонтные работы,Работы коммерческих организаций,NaN,29.05.2024,19.11.2024,1,line,"37.597433083289545,55.7834360473714;37.5972211...","{coordinates=[[[37.597433083, 55.783436047], [..."
4,20301,"ул. Маршала Тухачевского, д.15 - д.14, к.1",2639494385,Северо-Западный административный округ,район Хорошёво-Мнёвники,Ремонтные работы,Работы ДЖКХ,NaN,07.02.2024,30.06.2024,1,line,"37.48624339699745,55.78197922246133;37.4867811...","{coordinates=[[[37.486243397, 55.781979222], [..."


Первая строка датасета выступает в качестве описания датасета. Уберем ее из данных

In [ ]:
column_descriptions = df.iloc[0].tolist()
column_descriptions

['Идентификатор',
 'Местоположение',
 'global_id',
 'Административный округ',
 'Район',
 'Тип события',
 'Наименование события',
 'Инициатор',
 'Дата и время начала перекрытия',
 'Дата и время окончания перекрытия',
 'Количество перекрытых полос',
 'Тип геометрии',
 'Координаты для линейных объектов',
 'geoData']

In [ ]:
df = df.drop(0)
df.head()

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,Geometry_type,Geometry_line,geoData
1,20759,"Фрунзенская наб., д.8",2639121078,Центральный административный округ,район Хамовники,Ремонтные работы,Работы ДЖКХ,NaN,19.02.2024,20.05.2025,1,line,"37.595787444884735,55.73287012908498;37.596229...","{coordinates=[[[37.595787445, 55.732870129], [..."
2,20890,ул. Большая Академическая (ул. Валаамская - ул...,2639494326,Северный административный округ,Тимирязевский район,Ремонтные работы,Работы коммерческих организаций,NaN,15.01.2024,08.12.2024,1,line,"37.56010003387928,55.84447126979461;37.5600074...","{coordinates=[[[37.560100034, 55.84447127], [3..."
3,20659,ул. Новослободская (в сторону области) (ул. Па...,2639494348,Центральный административный округ,Тверской район,Ремонтные работы,Работы коммерческих организаций,NaN,29.05.2024,19.11.2024,1,line,"37.597433083289545,55.7834360473714;37.5972211...","{coordinates=[[[37.597433083, 55.783436047], [..."
4,20301,"ул. Маршала Тухачевского, д.15 - д.14, к.1",2639494385,Северо-Западный административный округ,район Хорошёво-Мнёвники,Ремонтные работы,Работы ДЖКХ,NaN,07.02.2024,30.06.2024,1,line,"37.48624339699745,55.78197922246133;37.4867811...","{coordinates=[[[37.486243397, 55.781979222], [..."
5,20292,"Зелёный пр-кт, д.76",2639494387,Восточный административный округ,район Новогиреево,Ремонтные работы,Работы ДЖКХ,NaN,19.10.2023,30.11.2024,1,line,"37.82988150918291,55.74968833030943;37.8305773...","{coordinates=[[[37.829881509, 55.74968833], [3..."


In [ ]:
descriptions_df = pd.DataFrame(column_descriptions, columns=['Descriptions'], index=df.columns)

print("Column Descriptions:")
print(descriptions_df)

Column Descriptions:
                                    Descriptions
Ext_id                             Идентификатор
Address                           Местоположение
global_id                              global_id
AdmArea                   Административный округ
District                                   Район
Event_type                           Тип события
Name                        Наименование события
Requester                              Инициатор
TS_from           Дата и время начала перекрытия
TS_to          Дата и время окончания перекрытия
Lanes_closed         Количество перекрытых полос
Geometry_type                      Тип геометрии
Geometry_line   Координаты для линейных объектов
geoData                                  geoData


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 1 to 1174
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Ext_id         1174 non-null   object
 1   Address        1174 non-null   object
 2   global_id      1174 non-null   object
 3   AdmArea        1174 non-null   object
 4   District       1174 non-null   object
 5   Event_type     1174 non-null   object
 6   Name           1174 non-null   object
 7   Requester      741 non-null    object
 8   TS_from        1174 non-null   object
 9   TS_to          1174 non-null   object
 10  Lanes_closed   512 non-null    object
 11  Geometry_type  1174 non-null   object
 12  Geometry_line  1174 non-null   object
 13  geoData        1174 non-null   object
dtypes: object(14)
memory usage: 128.5+ KB


In [ ]:
df.describe(include = "all")

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,Geometry_type,Geometry_line,geoData
count,1174,1174,1174,1174,1174,1174,1174,741,1174,1174,512,1174,1174,1174
unique,1174,686,1174,18,138,2,7,176,319,152,5,1,729,729
top,20759,ул. Большая Академическая (ул. Валаамская - ул...,2639121078,Центральный административный округ,Пресненский район,Ремонтные работы,Работы ДЖКХ,АО «Мосинжпроект»,11.11.2024,31.12.2024,1,line,"37.631952,55.684617;37.631153,55.68498;37.6305...","{coordinates=[[[37.631952, 55.684617], [37.631..."
freq,1,6,1,232,58,1173,481,66,25,335,388,1174,5,5


##**Заполнение пропусков**

In [ ]:
df.isna().sum()

,0
Ext_id,0
Address,0
global_id,0
AdmArea,0
District,0
Event_type,0
Name,0
Requester,433
TS_from,0
TS_to,0


In [ ]:
df['Requester'].fillna('Unknown', inplace=True)

In [ ]:
df['Lanes_closed'].fillna(0, inplace=True)

##**Изменение типов данных**

In [ ]:
df['Ext_id'].value_counts()

,count
Ext_id,
20759,1
169,1
147,1
148,1
149,1
...,...
47185,1
45508,1
47132,1


In [ ]:
df['Ext_id'] = df['Ext_id'].astype(np.int32)

In [ ]:
df['Lanes_closed'].value_counts()

,count
Lanes_closed,
0,662
1,388
2,105
4,10
3,8
5,1


In [ ]:
df['Lanes_closed'] = df[df['Lanes_closed'].notna()]['Lanes_closed'].astype(np.int32)

In [ ]:
df['TS_from'] = pd.to_datetime(df['TS_from'], format='%d.%m.%Y',  errors='coerce')
df['TS_to'] = pd.to_datetime(df['TS_to'], format='%d.%m.%Y',  errors='coerce')

In [ ]:
def extract_coordinates(geometry_str):
    pattern = r"coordinates=\[\[\[(.*?)\]\]"
    match = re.search(pattern, geometry_str)

    if match:
        coordinates_str = match.group(1)
        return f'[[{coordinates_str}]]'
    return None

df['geoData'] = df['geoData'].apply(extract_coordinates)

In [ ]:
df.head()

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,Geometry_type,Geometry_line,geoData
1,20759,"Фрунзенская наб., д.8",2639121078,Центральный административный округ,район Хамовники,Ремонтные работы,Работы ДЖКХ,Unknown,2024-02-19,2025-05-20,1,line,"37.595787444884735,55.73287012908498;37.596229...","[[37.595787445, 55.732870129], [37.59622944, 5..."
2,20890,ул. Большая Академическая (ул. Валаамская - ул...,2639494326,Северный административный округ,Тимирязевский район,Ремонтные работы,Работы коммерческих организаций,Unknown,2024-01-15,2024-12-08,1,line,"37.56010003387928,55.84447126979461;37.5600074...","[[37.560100034, 55.84447127], [37.560007415, 5..."
3,20659,ул. Новослободская (в сторону области) (ул. Па...,2639494348,Центральный административный округ,Тверской район,Ремонтные работы,Работы коммерческих организаций,Unknown,2024-05-29,2024-11-19,1,line,"37.597433083289545,55.7834360473714;37.5972211...","[[37.597433083, 55.783436047], [37.597221136, ..."
4,20301,"ул. Маршала Тухачевского, д.15 - д.14, к.1",2639494385,Северо-Западный административный округ,район Хорошёво-Мнёвники,Ремонтные работы,Работы ДЖКХ,Unknown,2024-02-07,2024-06-30,1,line,"37.48624339699745,55.78197922246133;37.4867811...","[[37.486243397, 55.781979222], [37.48678118, 5..."
5,20292,"Зелёный пр-кт, д.76",2639494387,Восточный административный округ,район Новогиреево,Ремонтные работы,Работы ДЖКХ,Unknown,2023-10-19,2024-11-30,1,line,"37.82988150918291,55.74968833030943;37.8305773...","[[37.829881509, 55.74968833], [37.830577344, 5..."


##**Очистка данных**

Посмотрим некоторые категориальные признаки руками





In [ ]:
df['AdmArea'].value_counts()

,count
AdmArea,
Центральный административный округ,232
Западный административный округ,138
Южный административный округ,136
Восточный административный округ,130
Северо-Восточный административный округ,112
Северный административный округ,107
Юго-Восточный административный округ,85
Северо-Западный административный округ,78
Юго-Западный административный округ,72


Разбиваем на несколько строк строки, где несколько районов

In [ ]:
df['AdmArea'] = df['AdmArea'].str.replace('[', '')
df['AdmArea'] = df['AdmArea'].str.replace(']', '')
df['AdmArea'] = df['AdmArea'].str.split(', ')
df = df.explode('AdmArea')
df['AdmArea'].value_counts()

,count
AdmArea,
Центральный административный округ,239
Западный административный округ,139
Южный административный округ,138
Восточный административный округ,130
Северо-Восточный административный округ,112
Северный административный округ,109
Юго-Восточный административный округ,88
Северо-Западный административный округ,78
Юго-Западный административный округ,73


In [ ]:
df['District'] = df['District'].str.replace('[', '')
df['District'] = df['District'].str.replace(']', '')
df['District'] = df['District'].str.split(', ')
df = df.explode('District')

Удаление дублирующихся строк

In [ ]:
# before = df.shape[0]
# df = df.drop_duplicates(subset=df.columns[df.columns != 'geoData'])
# after = df.shape[0]

# print(f"Удалено дублей: {before - after}")

Geometry_line повторяет признак geoData. Удалим Geometry_line

In [ ]:
df = df.drop(['Geometry_line'], axis=1)
df.head()

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,Geometry_type,geoData
1,20759,"Фрунзенская наб., д.8",2639121078,Центральный административный округ,район Хамовники,Ремонтные работы,Работы ДЖКХ,Unknown,2024-02-19,2025-05-20,1,line,"[[37.595787445, 55.732870129], [37.59622944, 5..."
2,20890,ул. Большая Академическая (ул. Валаамская - ул...,2639494326,Северный административный округ,Тимирязевский район,Ремонтные работы,Работы коммерческих организаций,Unknown,2024-01-15,2024-12-08,1,line,"[[37.560100034, 55.84447127], [37.560007415, 5..."
3,20659,ул. Новослободская (в сторону области) (ул. Па...,2639494348,Центральный административный округ,Тверской район,Ремонтные работы,Работы коммерческих организаций,Unknown,2024-05-29,2024-11-19,1,line,"[[37.597433083, 55.783436047], [37.597221136, ..."
4,20301,"ул. Маршала Тухачевского, д.15 - д.14, к.1",2639494385,Северо-Западный административный округ,район Хорошёво-Мнёвники,Ремонтные работы,Работы ДЖКХ,Unknown,2024-02-07,2024-06-30,1,line,"[[37.486243397, 55.781979222], [37.48678118, 5..."
5,20292,"Зелёный пр-кт, д.76",2639494387,Восточный административный округ,район Новогиреево,Ремонтные работы,Работы ДЖКХ,Unknown,2023-10-19,2024-11-30,1,line,"[[37.829881509, 55.74968833], [37.830577344, 5..."


Geometry_type не нужный признак

In [ ]:
df = df.drop(['Geometry_type'], axis=1)
df.head()

,Ext_id,Address,global_id,AdmArea,District,Event_type,Name,Requester,TS_from,TS_to,Lanes_closed,geoData
1,20759,"Фрунзенская наб., д.8",2639121078,Центральный административный округ,район Хамовники,Ремонтные работы,Работы ДЖКХ,Unknown,2024-02-19,2025-05-20,1,"[[37.595787445, 55.732870129], [37.59622944, 5..."
2,20890,ул. Большая Академическая (ул. Валаамская - ул...,2639494326,Северный административный округ,Тимирязевский район,Ремонтные работы,Работы коммерческих организаций,Unknown,2024-01-15,2024-12-08,1,"[[37.560100034, 55.84447127], [37.560007415, 5..."
3,20659,ул. Новослободская (в сторону области) (ул. Па...,2639494348,Центральный административный округ,Тверской район,Ремонтные работы,Работы коммерческих организаций,Unknown,2024-05-29,2024-11-19,1,"[[37.597433083, 55.783436047], [37.597221136, ..."
4,20301,"ул. Маршала Тухачевского, д.15 - д.14, к.1",2639494385,Северо-Западный административный округ,район Хорошёво-Мнёвники,Ремонтные работы,Работы ДЖКХ,Unknown,2024-02-07,2024-06-30,1,"[[37.486243397, 55.781979222], [37.48678118, 5..."
5,20292,"Зелёный пр-кт, д.76",2639494387,Восточный административный округ,район Новогиреево,Ремонтные работы,Работы ДЖКХ,Unknown,2023-10-19,2024-11-30,1,"[[37.829881509, 55.74968833], [37.830577344, 5..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1249 entries, 1 to 1174
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Ext_id        1249 non-null   int32         
 1   Address       1249 non-null   object        
 2   global_id     1249 non-null   object        
 3   AdmArea       1249 non-null   object        
 4   District      1249 non-null   object        
 5   Event_type    1249 non-null   object        
 6   Name          1249 non-null   object        
 7   Requester     1249 non-null   object        
 8   TS_from       1249 non-null   datetime64[ns]
 9   TS_to         1249 non-null   datetime64[ns]
 10  Lanes_closed  1249 non-null   int32         
 11  geoData       1249 non-null   object        
dtypes: datetime64[ns](2), int32(2), object(8)
memory usage: 117.1+ KB


In [ ]:
df.to_csv("Moscow_processed_data.csv", sep='^', index=False, encoding='utf-8')

## **Карта**



In [ ]:
m = folium.Map(location=[55.7558, 37.6173], zoom_start=10)  # Центр Москвы

# Добавление маркеров для каждого участка с дорожными работами
for idx, row in df.iterrows():
    coords = row['Geometry_line']  # Предполагается, что это строка с координатами
    if pd.notna(coords):
        coords = coords.split(';')[0].split(',')
        lat, lon = float(coords[1]), float(coords[0])
        folium.Marker(location=[lat, lon], popup=row['Address']).add_to(m)

# Сохранение карты в HTML файл
m.save('road_work_map.html')

In [ ]:
# def create_linestring(coords):
#     if pd.notna(coords):
#         # Заменяем ';' на ',' и разбиваем по ','
#         points = [tuple(map(float, point.split(','))) for point in coords.split(';')]
#         return LineString(points)
#     return None

# # Применение функции к столбцу Geometry_line
# df['geometry'] = df['Geometry_line'].apply(create_linestring)

# # Создание GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry='geometry')
# # Загрузка карты города (например, границы Москвы)
# # Вы можете использовать любой источник геоданных (например, shapefile)
# city_boundaries = gpd.read_file('path_to_city_boundaries.shp')

# # Построение карты
# fig, ax = plt.subplots(figsize=(12, 12))

# # Отображение границ города
# city_boundaries.plot(ax=ax, color='lightgrey', edgecolor='black')

# # Отображение дорожных работ
# gdf.plot(ax=ax, color='red', linewidth=2)

# # Настройка заголовка и отображение карты
# plt.title('Дорожные работы в Москве')
# plt.xlabel('Долгота')
# plt.ylabel('Широта')
# plt.show()

Здесь добавлены линии и выбор по административному *округу*. Вложенный выбор района внутри округа пока не получилось сделать (Можно посмотреть https://python-visualization.github.io/folium/latest/user_guide/plugins/featuregroup_subgroup.html)

In [ ]:
m = folium.Map(location=[55.7558, 37.6173], zoom_start=11)

# Словарь для хранения слоев по административным округам и районам
layer_groups = {}

# Проход по всем строкам DataFrame
for idx, row in df.iterrows():
    coords = row['Geometry_line']
    adm_area = row['AdmArea']
    district = row['District']

    if pd.notna(coords):
        # Преобразование строки координат в список точек
        points = [
            [float(lat), float(lon)]
            for lon, lat in (point.split(',') for point in coords.split(';'))
        ]

        # Создание слоя для административного округа, если его еще нет
        if adm_area not in layer_groups:
            layer_groups[adm_area] = {}

        # Создание слоя для района, если его еще нет
        if district not in layer_groups[adm_area]:
            layer_groups[adm_area][district] = folium.FeatureGroup(name=district)

        # Добавление линии в слой района
        folium.PolyLine(
            locations=points,
            color='blue',
            weight=3,
            popup=f"{row['Address']} ({row['Event_type']})"
        ).add_to(layer_groups[adm_area][district])

# Добавление всех слоев на карту
for adm_area, districts in layer_groups.items():
    # Создаем общий слой для административного округа
    adm_area_group = folium.FeatureGroup(name=adm_area)

    for district, layer in districts.items():
        adm_area_group.add_child(layer)  # Добавляем слой района в общий слой округа

    adm_area_group.add_to(m)  # Добавляем общий слой округа на карту

# Добавление переключателя слоев
folium.LayerControl().add_to(m)

# Сохранение карты
m.save('map_with_districts.html')

[[37.595787445, 55.732870129], [37.59622944, 55.733289645]]
[[37.560100034, 55.84447127], [37.560007415, 55.844435541]]
[[37.597433083, 55.783436047], [37.597221136, 55.783684344]]
[[37.486243397, 55.781979222], [37.48678118, 55.781894756], [37.487137248, 55.781819366], [37.48697564, 55.781857048]]
[[37.829881509, 55.74968833], [37.830577344, 55.74957679], [37.830598901, 55.749569617], [37.830077782, 55.749660571], [37.830283642, 55.749630003]]
[[37.58645609, 55.805691134], [37.586429328, 55.805770847]]
[[37.538083337, 55.536895779], [37.53847003, 55.536789545]]
[[37.602271736, 55.753597341], [37.602342255, 55.753632665]]
[[37.479076057, 55.762065318], [37.479354814, 55.762054466], [37.479171753, 55.762035602]]
[[37.55102478, 55.690822894], [37.550799223, 55.69069597], [37.55109787, 55.690864848]]
[[37.605758607, 55.757873674], [37.605312156, 55.757478305], [37.605578899, 55.757721239]]
[[37.583281435, 55.708195691], [37.583003417, 55.708038691], [37.582896247, 55.707976368]]
[[37.6111

Для датасета СПб можно добавить фильтр по дате

In [ ]:
# import folium
# from folium.plugins import TimestampedGeoJson
# import pandas as pd
# import json

# df = pd.DataFrame(data)

# # Конвертируем даты в тип datetime
# df['TS_from'] = pd.to_datetime(df['TS_from'])
# df['TS_to'] = pd.to_datetime(df['TS_to'])

# # Создание списка объектов GeoJSON с привязкой ко времени
# features = []
# for idx, row in df.iterrows():
#     coords = row['Geometry_line']
#     if pd.notna(coords):
#         # Преобразование строки координат в список точек
#         points = [
#             [float(lat), float(lon)]
#             for lon, lat in (point.split(',') for point in coords.split(';'))
#         ]

#         # Создание объекта GeoJSON для текущей строки
#         feature = {
#             "type": "Feature",
#             "geometry": {
#                 "type": "LineString",
#                 "coordinates": [[lon, lat] for lat, lon in points]
#             },
#             "properties": {
#                 "popup": f"{row['Address']} ({row['Event_type']})",
#                 "times": [row['TS_from'].isoformat(), row['TS_to'].isoformat()]
#             }
#         }
#         features.append(feature)

# # Создание слоя GeoJSON
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Создание карты
# m = folium.Map(location=[55.7558, 37.6173], zoom_start=11)

# # Добавление TimestampedGeoJson слоя
# TimestampedGeoJson(
#     data=geojson,
#     transition_time=500,  # Время перехода между объектами в миллисекундах
#     loop=False,           # Циклический просмотр времени
#     auto_play=False,      # Автоматический запуск
#     add_last_point=True   # Добавить последний видимый объект
# ).add_to(m)

# # Сохранение карты
# m.save('map_with_date_filter.html')
# print("Карта успешно сохранена в 'map_with_date_filter.html'.")